<a href="https://colab.research.google.com/github/patcharaponmai/CORALINE_Data_Engineer/blob/main/Take_Home_Coraline_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1
นำเข้าข้อมูลจากชีท FoodSales เข้า Database PostgreSQL โดยรวมข้อมูลทุกปีไว้ใน table เดียวกัน โดยมีข้อกำหนดของ PostgreSQL คือ


1.   database_name : challenge
2.   user: root
3.   password: DataEngineer_2023
4.   user: food_sales



In [1]:
# ทำการเรียก module pandas เพื่อใช้ในการอ่านไฟล์ de_challenge_data (1).xlsx

import pandas as pd

In [2]:
# กำหนด variable file_path ขึ้นมาเพื่อระบุตำแหน่ง source file ซึ่งถ้ามีการเปลี่ยนแปลงตำแหน่ง file จะได้แก้จาก variable ตัวนี้แทนได้เลย

file_path='/content/de_challenge_data.xlsx'
df = pd.read_excel(file_path, sheet_name='FoodSales', skiprows=[0]) # กำหนด skiprows=[0] เนื่องจาก cell แรกที่ระบุมาใน source file นั้นไม่จำเป็นต้องนำมาใช้

In [3]:
df.head(10)

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13 00:00:00,East,Boston,Cookies,Arrowroot,38,2.18,82.84
5,ID07356,2022-01-16 00:00:00,East,Boston,Bars,Carrot,54,1.77,95.58
6,ID07357,2022-01-19 00:00:00,East,Boston,Crackers,Whole Wheat,149,3.49,520.01
7,ID07358,2022-01-22 00:00:00,West,Los Angeles,Bars,Carrot,51,1.77,90.27
8,ID07359,2022-01-25 00:00:00,East,New York,Bars,Carrot,100,1.77,177
9,ID07360,2022-01-28 00:00:00,East,New York,Snacks,Potato Chips,28,1.35,37.8


In [4]:
# เช็ค information ของ data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          246 non-null    object
 1   Date        245 non-null    object
 2   Region      245 non-null    object
 3   City        245 non-null    object
 4   Category    245 non-null    object
 5   Product     245 non-null    object
 6   Qty         245 non-null    object
 7   UnitPrice   245 non-null    object
 8   TotalPrice  245 non-null    object
dtypes: object(9)
memory usage: 17.5+ KB


จะเห็นได้ว่า Column = 'ID' มีจำนวน record แตกต่างจาก column อื่นๆ จึงต้องทำการตรวจสอบว่ามีค่า Null, duplicate หรือมีค่าผิดจากปกติหรือไม่

In [5]:
df.isna().sum()

ID            1
Date          2
Region        2
City          2
Category      2
Product       2
Qty           2
UnitPrice     2
TotalPrice    2
dtype: int64

In [6]:
df.duplicated().sum()

0

จากการตรวจสอบค่า Null, duplicate พบว่า Column = 'ID' พบ Null จำนวน 1 record ในขณะที่ Column อื่นๆพบจำนวน 2 record จึงต้องทำการตรวจสอบ data และทำการ clean Null ออก

In [7]:
# เนื่องจากไม่รู้ว่า data ที่แตกต่างจาก record อื่นอยู๋ตำแหน่งไหน จึงจำเป็นต้อง sort data เพื่อเช็คดูครับ
# จึงได้เลือก Column = 'ID' ในการนำมา sort เพราะดูเป็น value ที่ unique สุดครับ
df['ID'] = df['ID'].astype('str')
df_sorted = df.sort_values(by='ID', ascending=True)

In [8]:
df_sorted

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
123,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
...,...,...,...,...,...,...,...,...,...
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1
246,ID07594,2023-12-30 00:00:00,West,Los Angeles,Cookies,Oatmeal Raisin,44,2.84,124.96


จาก result ข้างต้นจะเห็นว่ามี row ที่มี data ไม่เหมือน row อื่นๆ จึงจำเป็นต้อง clean row เหล่านี้ออก

In [9]:
# ทำการระบุ row ที่มี data เดียวกับ header

df.loc[df['ID'] == 'ID']

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
124,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice


In [10]:
# filter out row ที่เหมือนกับ header

df_filtered = df.loc[~(df['ID'] == 'ID')]

In [11]:
df_filtered.sort_values(by='ID')

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
123,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
...,...,...,...,...,...,...,...,...,...
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1
246,ID07594,2023-12-30 00:00:00,West,Los Angeles,Cookies,Oatmeal Raisin,44,2.84,124.96


In [12]:
# ทำการระบุ row ที่มี Null value

df_filtered[df_filtered.isna().any(axis=1)]

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
122,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# filter out row ที่มี Null value

df_cleaned = df_filtered[~(df_filtered.isna().any(axis=1))]

In [14]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 246
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          244 non-null    object
 1   Date        244 non-null    object
 2   Region      244 non-null    object
 3   City        244 non-null    object
 4   Category    244 non-null    object
 5   Product     244 non-null    object
 6   Qty         244 non-null    object
 7   UnitPrice   244 non-null    object
 8   TotalPrice  244 non-null    object
dtypes: object(9)
memory usage: 19.1+ KB


In [15]:
df_cleaned.isna().sum()

ID            0
Date          0
Region        0
City          0
Category      0
Product       0
Qty           0
UnitPrice     0
TotalPrice    0
dtype: int64

In [16]:
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])
df_cleaned['Date'] = df_cleaned['Date'].dt.strftime('%Y-%m-%d')

<ipython-input-16-a05e33663544>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])
<ipython-input-16-a05e33663544>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date'] = df_cleaned['Date'].dt.strftime('%Y-%m-%d')


In [17]:
df_cleaned.head(10)

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,2022-01-01,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13,East,Boston,Cookies,Arrowroot,38,2.18,82.84
5,ID07356,2022-01-16,East,Boston,Bars,Carrot,54,1.77,95.58
6,ID07357,2022-01-19,East,Boston,Crackers,Whole Wheat,149,3.49,520.01
7,ID07358,2022-01-22,West,Los Angeles,Bars,Carrot,51,1.77,90.27
8,ID07359,2022-01-25,East,New York,Bars,Carrot,100,1.77,177
9,ID07360,2022-01-28,East,New York,Snacks,Potato Chips,28,1.35,37.8


# Database connection

**รัน install แค่ครั้งเดียว**

In [18]:
# install PostgreSQL
!apt install postgresql postgresql-contrib &>log

# start PostgreSQL serviecs
!service postgresql start

# Create root user
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

# Create database
!sudo -u postgres createdb challenge

 * Starting PostgreSQL 14 database server
   ...done.
CREATE ROLE


In [19]:
# ทำการเรียก module psycopg2 เพื่อสร้าง connection ไปยัง database PostgreSQL

import psycopg2

In [20]:
# ประกาศ variable หลักที่จะใช้

DATABASE = "challenge"
USER = "root"
PASSWORD = "DataEngineer_2023"
TABLE_NAME = 'food_sales'

In [21]:
# สร้าง function สำหรับการ connect ไปยัง PostgreSQL database

def create_connection(Database : str, User : str, Password : str):
  conn = psycopg2.connect(
  database=Database,
  user=User,
  password=Password,
  )

  return conn

In [22]:
# สร้าง function เพื่อทำการ execute query ต่างๆ โดยจุดประสงค์หลักทำเพื่อดัก error ที่อาจจะเกิดเวลารัน query ที่เกี่ยวข้องกับ DELETE, UPDATE, INSERT จะได้ไม่ทำให้ data เสียหาย

def execute_query(QUERY):
  try:
    cur.execute(QUERY)
    conn.commit() # เรียกใช้ method commit สำหรับการ confirm การทำงานของ execution ก่อนหน้า
  except Exception as e:
    conn.rollback()
    print(f"Execute query Failed", e)

In [23]:
# สร้าง function ขึ้นมาเพื่อตรวจสอบ table ที่สร้างขึ้นมา

def check_ddl_table(TABLENAME):

  QUERY_CHECK_TBL = f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s
        ORDER BY ordinal_position
  """

  # ใช้การ substitute (%s) เพื่อง่ายต่อการเปลี่ยนแปลง TABLE_NAME ในอนาคต
  # เนื่องจากมีการใช้ substitute ในขั้นตอนการ execute จึงจำเป็นต้องใส่ agrument ที่ 2 เป็น tuple value เพื่อตอน execute จะได้ทำค่า TABLE_NAME นี้ parse ไปยัง '%s'
  cur.execute(QUERY_CHECK_TBL, (TABLENAME,))

  # Fetch the output if needed
  column_info = cur.fetchall()

  for row in column_info:
    print(row)

In [24]:
# เรียกใช้ method cursor สำหรับการ execute sql และ fetch data ที่จะใช้สำหรับการ query data
conn = create_connection(DATABASE, USER, PASSWORD)
cur = conn.cursor()

QUERY_CREAT_TBL = f""" CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
  "ID" VARCHAR(256),
  "Date" date,
  "Region" VARCHAR(256),
  "City" VARCHAR(256),
  "Category" VARCHAR(256),
  "Product" VARCHAR(256),
  "Qty" INTEGER,
  "UnitPrice" FLOAT,
  "TotalPrice" FLOAT
)
"""
execute_query(QUERY_CREAT_TBL)


In [25]:
check_ddl_table(TABLE_NAME)

('ID', 'character varying')
('Date', 'date')
('Region', 'character varying')
('City', 'character varying')
('Category', 'character varying')
('Product', 'character varying')
('Qty', 'integer')
('UnitPrice', 'double precision')
('TotalPrice', 'double precision')


In [26]:
# สร้าง function ขึ้นมาเพื่อทำการ ingest data

def Ingest_df_to_db(DataFrame : pd.DataFrame, Table_name : str, Database : str, User : str, PassWord : str, BatchSize : int = 1e2, Truncate = True) -> None:
  """
  Parameter
    1. DataFrame is parameter for pandas DataFrame
    2. Table_name is parameter for pandas table name
    3. Database is parameter for database name
    4. User is parameter for user
    5. PassWord is parameter for password
    6. BatchSize is size of batch to insert into table (default = 100 records)
    7. Truncate is delete all record from table before ingestion (default = True)

  """

  TOTAL_REC = len(DataFrame)


  # ทำการ truncate data ทุกครั้งเนื่องจากสมมุตว่ามี data แค่ชุดเดียวในการ ingest ซึ่งถ้าในอนาคตต้องมีการ insert data เข้าไปเพิ่ม จะต้องมาแก้ส่วนนี้
  if Truncate:
    try:
      print(f"START TRUNCATE TABLE `{Table_name}` ....")
      cur.execute(f"TRUNCATE TABLE {Table_name}")
      conn.commit()
      print(f"TRUNCATE TABLE `{Table_name}` >> COMPLETE")
    except Exception as e:
      print(f"TRUNCATE TABLE `{Table_name}` >> FAILED", e)

  print(f"======== START INGEST DATA INTO `{Database}`.`{Table_name}` ========")

  try:
    for index, row in DataFrame.iterrows():
      id=row['ID']
      date=row['Date']
      region=row['Region']
      city=row['City']
      category=row['Category']
      product=row['Product']
      qty=row['Qty']
      unitprice=row['UnitPrice']
      totalprice=row['TotalPrice']

      QUERY_INSERT = f""" INSERT INTO {Table_name} ("ID", "Date", "Region", "City", "Category", "Product", "Qty", "UnitPrice", "TotalPrice")
      VALUES ('{id}', '{date}', '{region}', '{city}', '{category}', '{product}', '{qty}', '{unitprice}', '{totalprice}') """

      execute_query(QUERY_INSERT)

      if (index % BatchSize == 1+0e-2 ) or ((index + 1) % TOTAL_REC ==  0):
        conn.commit()

        message : str = f"ROWS INSERT STATUS ------ [{index}/{TOTAL_REC}] ------"
        print(message)

    # Check record count in table
    cur.execute(f"SELECT COUNT(1) FROM {Table_name}")
    result_rec_count = cur.fetchall()

    if result_rec_count[0][0] == TOTAL_REC:
      rec_count_status = "SUCCESS"
    else:
      rec_count_status = "FAILED"

    # Check null in table

    QUERY_CHK_NULL = f"""
        SELECT COUNT(*)
        FROM {Table_name}
        WHERE
          "ID" IS NULL OR
          "Date" IS NULL OR
          "Region" IS NULL OR
          "City" IS NULL OR
          "Category" IS NULL OR
          "Product" IS NULL OR
          "Qty" IS NULL OR
          "UnitPrice" IS NULL OR
          "TotalPrice" IS NULL
    """

    cur.execute(QUERY_CHK_NULL)
    result_chk_null = cur.fetchall()


    if result_chk_null[0][0] == 0:
      chk_null_status = "SUCCESS"
    else:
      chk_null_status = "FAILED"

    print(f"======== RECONCILE PROCESS ========")
    print(f"Source rows = {TOTAL_REC}")
    print(f"Target rows = {result_rec_count[0][0]}")
    print(f"Record count status = {rec_count_status}")
    print(f"Check null status = {chk_null_status}")

  except Exception as e:
      raise("Something went wrong", e)

In [27]:
Ingest_df_to_db(df_cleaned, TABLE_NAME, 'challenge', 'root', 'DataEngineer_2023')

START TRUNCATE TABLE `food_sales` ....
TRUNCATE TABLE `food_sales` >> COMPLETE
======== START INGEST DATA INTO `challenge`.`food_sales` ========
ROWS INSERT STATUS ------ [1/244] ------
ROWS INSERT STATUS ------ [101/244] ------
ROWS INSERT STATUS ------ [201/244] ------
ROWS INSERT STATUS ------ [243/244] ------
======== RECONCILE PROCESS ========
Source rows = 244
Target rows = 244
Record count status = SUCCESS
Check null status = SUCCESS


In [28]:
# ทำการ select data ขึ้นมาดูว่าลง table เรียบร้อยแล้วหรือไม่ โดยใช้ return result ให้อยู่ในรูป DataFrame เพื่อสะดวกต่อการดูเนื่องจากมีหลาย column ครับ

cur.execute(f" SELECT * FROM {TABLE_NAME} LIMIT 5")
result = cur.fetchall()
columns = ['ID', 'Date', 'Region', 'City', 'Category', 'Product', 'Qty', 'UnitPrice', 'TotalPrice']
result_df = pd.DataFrame(result, columns=columns)
result_df

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,2022-01-01,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13,East,Boston,Cookies,Arrowroot,38,2.18,82.84


In [29]:
# Close connection
cur.close()
conn.close()

# TASK 2
เขียน SQL script เพื่อสร้าง table: cat_reg โดยใช้ data จาก Task1


## First solution : Create table

In [30]:
# ทำการเปิด connection อีกครั้ง
conn = create_connection(DATABASE, USER, PASSWORD)
cur = conn.cursor()

NEW_TABLE = 'cat_reg'

# ทำการสร้าง drop table ก่อนทุกครั้งเพื่อให้มั่นใจว่า table นี้ยังไม่เคยถูกสร้างมาก่อน
QUERY_DROP_TBL = f"DROP TABLE IF EXISTS {NEW_TABLE}"


# ทำการสร้าง table โดย datatype ในแต่ละ column ผมสมมุติให้ค่าเป็น integer โดยอ้างอิงจาก instruction ที่ให้มาครับ
QUERY_CREAT_NEW_TBL = f""" CREATE TABLE IF NOT EXISTS {NEW_TABLE} (
  "Category" VARCHAR(256),
  "East" INTEGER,
  "West" INTEGER,
  "Grand Total" INTEGER
)
"""

execute_query(QUERY_DROP_TBL)
execute_query(QUERY_CREAT_NEW_TBL)

In [31]:
check_ddl_table(NEW_TABLE)

('Category', 'character varying')
('East', 'integer')
('West', 'integer')
('Grand Total', 'integer')


In [32]:
QUERY_INSERT = f""" INSERT INTO {NEW_TABLE} ("Category", "East", "West", "Grand Total")
  SELECT DISTINCT("Category"),
  ROUND(SUM(CASE WHEN "Region" = 'East' THEN "TotalPrice" ELSE 0 END)) as total_price_east,
  ROUND(SUM(CASE WHEN "Region" = 'West' THEN "TotalPrice" ELSE 0 END)) as total_price_west,
  ROUND(SUM("TotalPrice"))
  FROM {TABLE_NAME}
  GROUP BY "Category"
  ORDER BY "Category"
"""

execute_query(QUERY_INSERT)

In [33]:
cur.execute(f" SELECT * FROM {NEW_TABLE} LIMIT 5")
conn.commit()
result = cur.fetchall()
columns = ['Category', 'East', 'West', 'Grand Total']
result_df = pd.DataFrame(result, columns=columns)
result_df = result_df.set_index('Category')
result_df


,East,West,Grand Total
Category,,,
Bars,6355,4180,10536
Cookies,10684,6529,17212
Crackers,3026,314,3340
Snacks,1460,778,2238


In [34]:
# Close connection
cur.close()
conn.close()

## Second Solution : Create view

In [35]:
# ทำการเปิด connection อีกครั้ง
conn = create_connection(DATABASE, USER, PASSWORD)
cur = conn.cursor()

NEW_TABLE_2 = 'cat_reg_2'

# ทำการสร้าง drop table ก่อนทุกครั้งเพื่อให้มั่นใจว่า table นี้ยังไม่เคยถูกสร้างมาก่อน
QUERY_DROP_VIEW_TBL = f"DROP VIEW IF EXISTS {NEW_TABLE_2}"

# ทำการสร้าง table โดย datatype ในแต่ละ column ผมสมมุติให้ค่าเป็น integer โดยอ้างอิงจาก instruction ที่ให้มาครับ
QUERY_CREAT_VIEW_TBL = f""" CREATE VIEW {NEW_TABLE_2} AS
  SELECT DISTINCT("Category") as Category,
  ROUND(SUM(CASE WHEN "Region" = 'East' THEN "TotalPrice" ELSE 0 END)) as East,
  ROUND(SUM(CASE WHEN "Region" = 'West' THEN "TotalPrice" ELSE 0 END)) as West,
  ROUND(SUM("TotalPrice")) as "Grand Total"
  FROM {TABLE_NAME}
  GROUP BY "Category"
  ORDER BY "Category"
"""

execute_query(QUERY_DROP_VIEW_TBL)
execute_query(QUERY_CREAT_VIEW_TBL)

In [36]:
cur.execute(f" SELECT * FROM {NEW_TABLE_2} LIMIT 5")
result_2 = cur.fetchall()
columns = ['Category', 'East', 'West', 'Grand Total']
result_df_2 = pd.DataFrame(result_2, columns=columns)
result_df_2 = result_df_2.set_index('Category')
result_df_2

,East,West,Grand Total
Category,,,
Bars,6355.0,4180.0,10536.0
Cookies,10684.0,6529.0,17212.0
Crackers,3026.0,314.0,3340.0
Snacks,1460.0,778.0,2238.0


In [37]:
# Close connection
cur.close()
conn.close()

# SQL script

Command create table cat_reg

```
# While creating a table, column name should enclose with "" to avoid an issue. For instance, reserved word, space in column name, case sensitive and etc.

CREATE TABLE IF NOT EXISTS cat_reg (
  "Category" VARCHAR(256),
  "East" INTEGER,
  "West" INTEGER,
  "Grand Total" INTEGER
);
```

Command insert data into cat_reg

```
INSERT INTO cat_reg ("Category", "East", "West", "Grand Total")
  SELECT DISTINCT("Category"),
  ROUND(SUM(CASE WHEN "Region" = 'East' THEN "TotalPrice" ELSE 0 END)) as total_price_east,
  ROUND(SUM(CASE WHEN "Region" = 'West' THEN "TotalPrice" ELSE 0 END)) as total_price_west,
  ROUND(SUM("TotalPrice"))
  FROM {TABLE_NAME}
  GROUP BY "Category"
  ORDER BY "Category";
```

# SQL script for view table

```
CREATE VIEW cat_reg AS
  SELECT DISTINCT("Category") as Category,
  ROUND(SUM(CASE WHEN "Region" = 'East' THEN "TotalPrice" ELSE 0 END)) as East,
  ROUND(SUM(CASE WHEN "Region" = 'West' THEN "TotalPrice" ELSE 0 END)) as West,
  ROUND(SUM("TotalPrice")) as "Grand Total"
  FROM {TABLE_NAME}
  GROUP BY "Category"
  ORDER BY "Category";
```
